In [ ]:
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# Question a)

In [ ]:
def ICV_LBPimage(input_image):
    M, N = input_image.shape
    # Create a copy of the input image to store the LBP result
    lbp_image = input_image.copy()

    # Pad the input image using mirroring to handle border pixels
    padding_image = np.pad(input_image, ((1, 1), (1, 1)), mode='reflect')

    # Define the weights for LBP computation
    weights = np.array([[1, 2, 4], [128, 0, 8], [64, 32, 16]], dtype=np.uint8)

    # Make LBP processing for each pixel of the original image
    for i in range(1, M + 1):
        for j in range(1, N + 1):
            # Extract a 3x3 window with the current pixel as the center
            temp = padding_image[i - 1:i + 2, j - 1:j + 2]
            # Compare each pixel in the window with the center pixel and convert the result to binary (0 or 1)
            temp = (temp > temp[1, 1]).astype(np.uint8)
            # Calculate the LBP feature using the weights
            feature = np.sum(temp * weights)
            
            lbp_image[i - 1, j - 1] = feature

    return lbp_image

In [ ]:
# the feature descriptor for each window as distribution of LBP codes
def ICV_featureDescriptor(input_lbp_block):
    H, W = input_lbp_block.shape
    feature_descriptor = np.zeros(256)

    # Iterate over each possible LBP code (number 0 to 255)
    for number in range(256):
        # Iterate through each pixel in the LBP block
        for i in range(H):
            for j in range(W):
                # Check if the LBP code at the current pixel matches the current number
                if input_lbp_block[i, j] == number:
                    # Increment the corresponding bin in the feature descriptor
                    feature_descriptor[number] += 1
    return feature_descriptor

In [ ]:
# Normalize the histogram (which is now a feature descriptor representing the window).
def ICV_normalizeHistogram(input_histogram):
    M = input_histogram.shape[0]
    normalized_histogram = np.zeros((M))
    
    # Check if the input histogram is a 2D array (contains multiple histograms)
    if len(input_histogram.shape) >= 2:
        N = input_histogram.shape[1]
        normalized_histogram = np.zeros((M, N))

    # Check if the input histogram is a 1D array
    if len(input_histogram.shape) == 1:
        row_sum = np.sum(input_histogram)
        normalized_histogram = input_histogram / row_sum
    else:
        # Iterate over each row in the input histogram (2D case)
        for i in range(M):
            row_sum = np.sum(input_histogram[i, :])
            normalized_histogram[i, :] = input_histogram[i, :] / row_sum

    return normalized_histogram

In [ ]:
def ICV_lbp(inputImage, blocks_h, blocks_w):
    height, width = inputImage.shape
    lbp_image = ICV_LBPimage(inputImage) # Compute the Local Binary Pattern (LBP) image based on the input grayscale image.

    num_blocks = blocks_h * blocks_w
    block_size_h = round(height / blocks_h)
    block_size_w = round(width / blocks_w)
    feature_descriptors = np.zeros((num_blocks, 256))

    # Iterate over each block in the image
    for i in range(1, blocks_h + 1):
        for j in range(1, blocks_w + 1):
             # Extract the LBP features for the current block
            windowBlock = lbp_image[(i - 1) * block_size_h:i * block_size_h, (j - 1) * block_size_w:j * block_size_w]
            
            # Map the feature descriptor of the current block to the appropriate row in feature_descriptors.
            # The blocks_h factor is used to ensure correct indexing.
            feature_descriptors[i - 1 + (j - 1) * blocks_h, :] = ICV_featureDescriptor(windowBlock)  # Use the function to compute the feature descriptor (histogram) for the block.
   
    NormailzedFeatureDiscriptors = ICV_normalizeHistogram(feature_descriptors)

In [ ]:
# divides a greyscale image into equally sized non-overlapping windows
def ICV_divide_windows(input_image, num_window_H, num_window_W):
    H, W = input_image.shape
    window_size_H = H // num_window_H
    window_size_W = W // num_window_W
    window_cells = []

    for i in range(num_window_H):
        row_cell=[]
        for j in range(num_window_W):
            # Extract the block (window) from the input image
            block = input_image[i * window_size_H:(i + 1) * window_size_H, j * window_size_W:(j + 1) * window_size_W]
            row_cell.append(block)
        window_cells.append(row_cell)

    return np.array(window_cells)

img = cv2.imread('face-3.jpg')
image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

num_window_H = 4
num_window_W = 4
WindowCells = ICV_divide_windows(image, num_window_H, num_window_W)

In [ ]:
w1 = WindowCells[0,1]
plt.imshow(w1, cmap='gray')
plt.axis('off')

In [ ]:
w2 = WindowCells[2,1]
plt.imshow(w2, cmap='gray')
plt.axis('off')

In [ ]:
w3 = WindowCells[3,3]
plt.imshow(w3, cmap='gray')
plt.axis('off')

In [ ]:
# LBP of windows:
LBP_w1 = ICV_LBPimage(w1);
plt.imshow(LBP_w1, cmap='gray')
plt.axis('off')

In [ ]:
LBP_w2 = ICV_LBPimage(w2);
plt.imshow(LBP_w2, cmap='gray')
plt.axis('off')

In [ ]:
LBP_w3 = ICV_LBPimage(w3);
plt.imshow(LBP_w3, cmap='gray')
plt.axis('off')

In [ ]:
# Histograms of LBPs
h1_LBP = ICV_featureDescriptor(LBP_w1);
nor_h1_LBP = ICV_normalizeHistogram(h1_LBP);
plt.bar(range(len(nor_h1_LBP)), nor_h1_LBP)
plt.title('Normalized LBP Feature Descriptor')
plt.xlabel('LBP Code')
plt.ylabel('Normalized Frequency')

In [ ]:
h2_LBP = ICV_featureDescriptor(LBP_w2);
nor_h2_LBP = ICV_normalizeHistogram(h2_LBP);
plt.bar(range(len(nor_h2_LBP)), nor_h2_LBP)
plt.title('Normalized LBP Feature Descriptor')
plt.xlabel('LBP Code')
plt.ylabel('Normalized Frequency')

In [ ]:
h3_LBP = ICV_featureDescriptor(LBP_w3);
nor_h3_LBP = ICV_normalizeHistogram(h3_LBP);
plt.bar(range(len(nor_h3_LBP)), nor_h3_LBP)
plt.title('Normalized LBP Feature Descriptor')
plt.xlabel('LBP Code')
plt.ylabel('Normalized Frequency')

In [ ]:
# a function that divides a greyscale image into equally sized non-overlapping windows and returns the feature descriptor for each window as distribution of LBP codes.
def ICV_windowFeatureDescriptor(image, num_blocks_H, num_blocks_W):
    # Divide the image into windows
    WindowCells = ICV_divide_windows(image, num_blocks_H, num_blocks_W)
    
    # Initialize an array to store feature descriptors for each window
    feature_descriptors = []

    # Iterate through each window
    for i in range(num_blocks_H):
        for j in range(num_blocks_W):
            # Get the current window
            window = WindowCells[i, j]

            # Calculate LBP feature descriptor for the window
            window_lbp_descriptor = ICV_featureDescriptor(window)
            
            # Append the LBP descriptor to the list
            feature_descriptors.append(window_lbp_descriptor)

    return feature_descriptors

In [ ]:
# Question b)

In [ ]:
# a descriptor that represents the whole image as consisting of multiple windows
def ICV_descriptorOfTheWholeImage(input_image, num_blocks_H, num_blocks_W):
    # Divide the image into windows
    WindowCells = ICV_divide_windows(input_image, num_blocks_H, num_blocks_W)

    num_blocks = num_blocks_H * num_blocks_W
    feature_descriptors = np.zeros((num_blocks, 256))

    for i in range(num_blocks_H):
        for j in range(num_blocks_W):
            window = WindowCells[i][j]
            lbp_block = ICV_LBPimage(window)
            feature_descriptors[i + j * num_blocks_H, :] = ICV_featureDescriptor(lbp_block)

    # Normalize the feature descriptors
    normalized_feature_descriptors = ICV_normalizeHistogram(feature_descriptors)

    return normalized_feature_descriptors

In [ ]:
# Plot histograms
def plotDescriptor(image):
    num_blocks_H=4
    num_blocks_W=4
    descriptor1 = ICV_descriptorOfTheWholeImage(image, num_blocks_H, num_blocks_W)
    X = np.arange(256)
    # Set a common initial color
    for i in range(descriptor1.shape[0]):
        # Plot the bar chart
        plt.bar(X * i, descriptor1[i, :])
    plt.show()

# face-3.jpg
plotDescriptor(image)

In [ ]:
# car-2.jpg
car_img = cv2.imread('car-2.jpg')
image = cv2.cvtColor(car_img, cv2.COLOR_BGR2GRAY)
plotDescriptor(image)

In [ ]:
# implement a classification process that separates the images in the dataset into two categories:
def ICV_classifier(label_descriptor1, label_descriptor2, sample):
    # Get the dimensions of label descriptors
    M1, N1 = label_descriptor1.shape
    M2, _ = label_descriptor2.shape

    # Check if the sizes of label descriptors are equal
    if M1 != M2:
        print('The size of the two label descriptors should be equal!!!')
        return

    # Initialize distances
    distance1 = 0
    distance2 = 0

    # Calculate distances
    for i in range(M1):
        for j in range(N1):
            variance1 = np.sqrt((label_descriptor1[i, j] - sample[i, j])**2)
            distance1 += variance1

            variance2 = np.sqrt((label_descriptor2[i, j] - sample[i, j])**2)
            distance2 += variance2

    # Determine the output class
    output = 1 if distance1 < distance2 else 2

    return output, distance1, distance2

In [ ]:
import os

def imread(file_path):
    return cv2.imread(file_path)

def ICV_rgb2grayscale(image):
    if len(image.shape) == 3 and image.shape[2] == 3:
        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        # The image is already grayscale or has an invalid number of channels
        return image

def ICV_importFiles(sample_path, file_ext):
    num_data = 0
    test_dataset = []
    print('data_set:',os.listdir(sample_path))
    # Iterate through files in the directory
    for file in os.listdir(sample_path):
        # Check if the file has the specified extension
        if file.endswith(file_ext):
            # Read and process the image
            image = imread(os.path.join(sample_path, file))
            image = ICV_rgb2grayscale(image)
            test_dataset.append(image)
            num_data += 1
    return test_dataset, num_data

# Load images
face1 = imread('DatasetA/face-1.jpg')
face1 = ICV_rgb2grayscale(face1)
descriptor_face_1 = ICV_descriptorOfTheWholeImage(face1, 4, 4)

car1 = imread('DatasetA/car-1.jpg')
car1 = ICV_rgb2grayscale(car1)
descriptor_car_1 = ICV_descriptorOfTheWholeImage(car1, 4, 4)

# Label the two classes -- FACE & CAR
class_face = descriptor_face_1
class_car = descriptor_car_1

# Import images
sample_path = 'DatasetA/'
file_ext = '.jpg'
Test_dataset, numdata = ICV_importFiles(sample_path, file_ext)

Confidence = np.zeros(numdata)
TP, TN, FP, FN = 0, 0, 0, 0

# Print the prediction based on the classifier output
for i in range(numdata):
    grey_image = Test_dataset[i]
    sample = ICV_descriptorOfTheWholeImage(grey_image, 4, 4)
    
    # Use the classifier to make a prediction and obtain distances
    output, distance1, distance2 = ICV_classifier(class_face, class_car, sample)
    if output == 1:
        print('Prediction: The input sample belongs to the first class FACE.')
        if i <= 3:
            TP += 1
        else:
            FP += 1
    else:
        print('Prediction: The input sample belongs to the second class CAR.')
        if i <= 3:
            FN += 1
        else:
            TN += 1

# Calculate accuracy for the current window size
accuracy = (TP + TN) / (TP + FP + FN + TN)
print('classification accuracy:', accuracy)

In [ ]:
# question c) and question d)

In [ ]:
# Decrease and Increase the window size and perform classification again. 

# Numbers of windows to be tested
numbers_window = [1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196]

# Calculate square root of window numbers
n_sqrt = np.sqrt(numbers_window)

# Initialize arrays for accuracy and confidence
accuracy = np.zeros(len(numbers_window))
confidence = np.zeros((numdata, len(numbers_window)))

for i in range(len(numbers_window)):
    window_size = int(n_sqrt[i])

    class_Car = ICV_descriptorOfTheWholeImage(car1, window_size, window_size)
    class_Face = ICV_descriptorOfTheWholeImage(face1, window_size, window_size)

     # Initialize counters for evaluation metrics
    numPredictedCar = 0
    numPredictedFace = 0
    TP, TN, FP, FN = 0, 0, 0, 0

    # Loop through the test dataset
    for j in range(numdata):
        image = Test_dataset[j]
        sample = ICV_descriptorOfTheWholeImage(image, window_size, window_size)
        
        # Make a prediction using the classifier
        prediction, distance_Car, distance_Face = ICV_classifier(class_Car, class_Face, sample)

        # Update counters and confidence values based on the prediction
        if prediction == 1:
            numPredictedCar += 1
            if j <= 3:
                TP += 1
            else:
                FP += 1
        else:
            numPredictedFace += 1
            if j <= 3:
                FN += 1
            else:
                TN += 1
    # Calculate accuracy for the current window size
    accuracy[i] = (TP + TN) / (TP + FP + FN + TN)

plt.figure()
plt.plot(numbers_window, accuracy, '-o')
plt.title('Classification Evaluation')
plt.xlabel('Window Number')
plt.ylabel('Accuracy')
plt.show()